In [0]:
! nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!pwd

/content


In [0]:
cd gdrive/My\ Drive/Mask_Rcnn

/content/gdrive/My Drive/Mask_Rcnn


In [0]:
!ls -l | grep "bg"

drwx------ 2 root root       4096 May 10 10:13 bg
-rw------- 1 root root 1443877383 May 12 16:55 dataset_fg_bg_correct
-rw------- 1 root root 2289574765 May 10 10:01 dataset_fg_bg_final
-rw------- 1 root root  716783927 May 10 16:23 dataset_fg_bg_Mask_final
-rw------- 1 root root 1443877383 May 12 16:55 fg_bg_correct


In [0]:
!ls -la | grep "depth"

-rw------- 1 root root  397838790 May 13 02:47 depth_1.zip
-rw------- 1 root root  397955917 May 13 02:48 depth_2.zip
-rw------- 1 root root  398080862 May 13 02:49 depth_3.zip
-rw------- 1 root root  397839274 May 13 02:49 depth_4.zip
-rw------- 1 root root  398339753 May 13 02:49 depth_5.zip


In [3]:
from zipfile import ZipFile
import zipfile
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import io
from itertools import groupby
import cv2
from tqdm.auto import tqdm
from pathlib import Path
from time import time
from torchvision import datasets
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.utils.data import random_split
from PIL import Image
import torchvision.transforms as T
import os
import torch
sns.set()

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Let's see if the dataset works

In [0]:
#To import zip file
from zipfile import ZipFile 
  
# specifying the zip file name 
file_name = "/content/gdrive/My Drive/Mask_Rcnn/depth_5.zip"
  
# opening the zip file in READ mode 
with ZipFile(file_name, 'r') as zip: 
    zip.printdir() 

    print('Extracting all the files now...') 
    zip.extractall() 
    print('Done!')

Streaming output truncated to the last 5000 lines.
mask_fg_bg/fg_bg/mask_fg_015_bg_085_340600.png 2020-05-13 02:11:14         3879
mask_fg_bg/fg_bg/mask_fg_078_bg_075_303142.png 2020-05-13 02:11:14         6571
mask_fg_bg/fg_bg/mask_fg_035_bg_066_265404.png 2020-05-13 02:11:14         6079
mask_fg_bg/fg_bg/mask_fg_084_bg_097_391389.png 2020-05-13 02:11:14         5893
mask_fg_bg/fg_bg/mask_fg_073_bg_031_126925.png 2020-05-13 02:11:14         4345
mask_fg_bg/fg_bg/mask_fg_008_bg_070_280336.png 2020-05-13 02:11:14         4083
mask_fg_bg/fg_bg/mask_fg_074_bg_044_178965.png 2020-05-13 02:11:14         3305
mask_fg_bg/fg_bg/mask_fg_001_bg_091_364049.png 2020-05-13 02:11:14         6464
mask_fg_bg/fg_bg/mask_fg_015_bg_058_232613.png 2020-05-13 02:11:14         5843
mask_fg_bg/fg_bg/mask_fg_069_bg_042_170799.png 2020-05-13 02:11:14         5497
mask_fg_bg/fg_bg/mask_fg_051_bg_075_302077.png 2020-05-13 02:11:14         7562
mask_fg_bg/fg_bg/mask_fg_098_bg_036_147942.png 2020-05-13 02:11:14   

In [0]:
#Export Zip file
from zipfile import ZipFile 
import zipfile
import os 
  
def get_all_file_paths(directory): 
  
    # initializing empty file paths list 
    file_paths = [] 
  
    # crawling through directory and subdirectories 
    for root, directories, files in os.walk(directory): 
        for filename in files: 
            # join the two strings in order to form the full filepath. 
            filepath = os.path.join(root, filename) 
            file_paths.append(filepath) 
  
    # returning all file paths 
    return file_paths         
  
def main(): 
    # path to folder which needs to be zipped 
    directory = 'mask_fg_bg/fg_bg/'
  
    # calling function to get all file paths in the directory 
    file_paths = get_all_file_paths(directory) 
  
    # printing the list of all files to be zipped 
    print('Following files will be zipped:') 
    for file_name in file_paths: 
        print(file_name) 
  
    # writing files to a zipfile 
    with ZipFile('/content/gdrive/My Drive/Mask_Rcnn/mask_final','a', compression = zipfile.ZIP_DEFLATED) as zip: 
        # writing each file one by one 
        for file in file_paths: 
            zip.write(file) 
  
    print('All files zipped successfully!')         
  
  

main()

Streaming output truncated to the last 5000 lines.
mask_fg_bg/fg_bg/mask_fg_012_bg_010_040502.png
mask_fg_bg/fg_bg/mask_fg_088_bg_054_219555.png
mask_fg_bg/fg_bg/mask_fg_018_bg_091_364746.png
mask_fg_bg/fg_bg/mask_fg_003_bg_040_160121.png
mask_fg_bg/fg_bg/mask_fg_027_bg_078_313093.png
mask_fg_bg/fg_bg/mask_fg_001_bg_039_156040.png
mask_fg_bg/fg_bg/mask_fg_099_bg_010_043965.png
mask_fg_bg/fg_bg/mask_fg_089_bg_081_327586.png
mask_fg_bg/fg_bg/mask_fg_099_bg_037_151986.png
mask_fg_bg/fg_bg/mask_fg_018_bg_066_264722.png
mask_fg_bg/fg_bg/mask_fg_032_bg_018_073282.png
mask_fg_bg/fg_bg/mask_fg_019_bg_011_044769.png
mask_fg_bg/fg_bg/mask_fg_091_bg_026_107675.png
mask_fg_bg/fg_bg/mask_fg_065_bg_070_282629.png
mask_fg_bg/fg_bg/mask_fg_042_bg_089_357716.png
mask_fg_bg/fg_bg/mask_fg_070_bg_013_054806.png
mask_fg_bg/fg_bg/mask_fg_070_bg_080_322813.png
mask_fg_bg/fg_bg/mask_fg_079_bg_003_015194.png
mask_fg_bg/fg_bg/mask_fg_028_bg_025_101120.png
mask_fg_bg/fg_bg/mask_fg_060_bg_049_198400.png
mask_fg_b

In [0]:
depth_fg_bg_zip =  ZipFile('/content/gdrive/My Drive/Mask_Rcnn/mask_final', 'r')

In [0]:
all_files = [info.filename for info in depth_fg_bg_zip.infolist() if not info.is_dir()]

In [7]:
len(all_files)

400000

In [8]:
!ls -l '/content/gdrive/My Drive/Mask_Rcnn/' | grep "bg"

drwx------ 2 root root       4096 May 10 10:13 bg
-rw------- 1 root root   49453404 May 10 10:14 bg.zip
-rw------- 1 root root 1443877383 May 12 16:55 dataset_fg_bg_correct
-rw------- 1 root root 2289574765 May 10 10:01 dataset_fg_bg_final
-rw------- 1 root root  716783927 May 10 16:23 dataset_fg_bg_Mask_final
-rw------- 1 root root 1443877383 May 12 16:55 fg_bg_correct


In [0]:
fg_bg_zip =  ZipFile('/content/gdrive/My Drive/Mask_Rcnn/dataset_fg_bg_Mask_final', 'r')

In [0]:
all_files = [info.filename for info in fg_bg_zip.infolist() if not info.is_dir()]

In [11]:
len(all_files)

400000

In [0]:
class DenseDepth(Dataset):
    '''
    DenseDepth Dataset

    Input is fg_bg image AND bg image
    Target is fg_bg_mask AND depth_fg_bg image
    '''
    def __init__(self, root, data='fg_bg', train=True, transform=None, target_transform=None, download=False):
        self.root = root
        self.transform = transform
        self.target_transform = target_transform

        # open the respective data and target
        bg_zip = ZipFile(os.path.join(self.root,'/content/gdrive/My Drive/Mask_Rcnn/bg.zip'), 'r') # bg
        fg_bg_zip =  ZipFile(os.path.join(self.root,'/content/gdrive/My Drive/Mask_Rcnn/dataset_fg_bg_correct'), 'r') # fg_bg
        fg_bg_mask_zip = ZipFile(os.path.join(self.root,'/content/gdrive/My Drive/Mask_Rcnn/dataset_fg_bg_Mask_final'), 'r') # fg_bg_mask
        depth_fg_bg_zip =  ZipFile(os.path.join(self.root,'/content/gdrive/My Drive/Mask_Rcnn/mask_final'), 'r') # target

        bg_paths = [info.filename for info in bg_zip.infolist() if not info.is_dir()]
        fg_bg_paths = [info.filename for info in fg_bg_zip.infolist() if not info.is_dir()]
        fg_bg_mask_paths = [info.filename for info in fg_bg_mask_zip.infolist() if not info.is_dir()]
        depth_fg_bg_paths = [info.filename for info in depth_fg_bg_zip.infolist() if not info.is_dir()]


        assert(len(bg_paths) == 100)
        assert(len(fg_bg_paths) == 400000)
        assert(len(fg_bg_mask_paths) == 400000)
        assert(len(depth_fg_bg_paths) == 400000)

        print(f'found {len(bg_paths)} bg images, {len(fg_bg_paths)} fg_bg images, {len(fg_bg_mask_paths)} fg_bg_mask images, {len(depth_fg_bg_paths)} depth_fg_bg images')


        if data == 'fg_bg':
            self.image_zip = fg_bg_zip
            self.image_paths = fg_bg_paths
        elif data == 'bg':
            self.image_zip = bg_zip
            self.image_paths = bg_paths
        elif data == 'fg_bg_mask':
            self.image_zip = fg_bg_mask_zip
            self.image_paths = fg_bg_mask_paths
        elif data == 'depth_fg_bg':
            self.image_zip = depth_fg_bg_zip
            self.image_paths = depth_fg_bg_paths
        else:
            raise f'{data} is not a valid option'

        self.targets = []

    def __getitem__(self, index):
        imgdata = self.image_zip.read(self.image_paths[index])
        img = Image.open(io.BytesIO(imgdata))
        img = img.convert("RGB")
        img = np.array(img)

        if self.transform is not None:
            img = self.transform(img)
        
        return img

    def __len__(self):
        return len(self.image_paths)

In [0]:
del depth_dataset, depth_dataloader

In [13]:
depth_dataset = DenseDepth(root='./', transform=T.Compose([T.ToTensor()]))

found 100 bg images, 400000 fg_bg images, 400000 fg_bg_mask images, 400000 depth_fg_bg images


In [0]:
depth_dataloader = DataLoader(depth_dataset, batch_size=4, shuffle=True, num_workers=1)

In [0]:
def get_mean_and_std(dataset):
    '''Compute the mean and std value of dataset.'''
    dataloader = torch.utils.data.DataLoader(
        dataset, batch_size=1, shuffle=True, num_workers=1)
    mean = torch.zeros(3)
    std = torch.zeros(3)
    print('==> Computing mean and std..')
    for images in tqdm(dataloader):
        for i in range(3):
            mean[i] += images[:, i, :, :].mean()
            std[i] += images[:, i, :, :].std()
    mean.div_(len(dataset))
    std.div_(len(dataset))
    return mean, std

## Compute Mean and Std dev. for BG images

In [16]:
bg_dataset = DenseDepth(root='./', data='bg', transform=T.Compose([T.ToTensor()]))
mean, std = get_mean_and_std(bg_dataset)

found 100 bg images, 400000 fg_bg images, 400000 fg_bg_mask images, 400000 depth_fg_bg images
==> Computing mean and std..


In [17]:
[f'{m:.15f}' for m in mean], [f'{s:.15f}' for s in std]

(['0.445187151432037', '0.453919172286987', '0.442775547504425'],
 ['0.209829792380333', '0.204434618353844', '0.218459352850914'])

## Compute Mean and Std dev. for fg_bg images

In [18]:
fg_bg_dataset = DenseDepth(root='./', data='fg_bg', transform=T.Compose([T.ToTensor()]))
mean, std = get_mean_and_std(fg_bg_dataset)

found 100 bg images, 400000 fg_bg images, 400000 fg_bg_mask images, 400000 depth_fg_bg images
==> Computing mean and std..


In [19]:
[f'{m:.15f}' for m in mean], [f'{s:.15f}' for s in std]

(['0.458231598138809', '0.458402276039124', '0.454385221004486'],
 ['0.216196656227112', '0.213248178362846', '0.224730432033539'])

## Compute Mean and Std dev. for fg_bg_mask

In [20]:
fg_bg_mask_dataset = DenseDepth(root='./', data='fg_bg_mask', transform=T.Compose([T.ToTensor()]))
mean, std = get_mean_and_std(fg_bg_mask_dataset)

found 100 bg images, 400000 fg_bg images, 400000 fg_bg_mask images, 400000 depth_fg_bg images
==> Computing mean and std..


In [21]:
[f'{m:.15f}' for m in mean], [f'{s:.15f}' for s in std]

(['0.246712774038315', '0.246712774038315', '0.246712774038315'],
 ['0.419586002826691', '0.419586002826691', '0.419586002826691'])

## Compute Mean and Std dev. for depth_fg_bg

In [22]:
depth_fg_bg_dataset = DenseDepth(root='./', data='depth_fg_bg', transform=T.Compose([T.ToTensor()]))
mean, std = get_mean_and_std(depth_fg_bg_dataset)

found 100 bg images, 400000 fg_bg images, 400000 fg_bg_mask images, 400000 depth_fg_bg images
==> Computing mean and std..


In [24]:
[f'{m:.15f}' for m in mean], [f'{s:.15f}' for s in std]

(['0.180530488491058', '0.180530488491058', '0.180530488491058'],
 ['0.051506847143173', '0.051506847143173', '0.051506847143173'])